In [ ]:
import subprocess, os, sys, time, importlib

os.chdir('/workspaces/scrapermeleegg')
sys.path.insert(0, '/workspaces/scrapermeleegg')

# Kill any existing process on port 8080
subprocess.run('kill $(lsof -t -i:8080) 2>/dev/null', shell=True)
time.sleep(1)

# Regenerate the site
import generate_site
importlib.reload(generate_site)
import manage_tournaments as mt
importlib.reload(mt)

db = mt.TournamentDB()
dist_path = '/workspaces/scrapermeleegg/dist'
if db.tournaments:
    generate_site.generate_multi_tournament_site(db.data['tournaments'], dist_path)
    size = os.path.getsize(os.path.join(dist_path, 'index.html'))
    print(f'Site generated: {size} bytes')

with open(os.path.join(dist_path, 'index.html'), 'r') as f:
    html = f.read()
for term in ['home-panel', 'add-box', 'Scrapear', 'btn-do-add']:
    print(f'  {term}: {html.count(term)}')

server = subprocess.Popen(
    [sys.executable, 'server.py', '--port', '8080'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
    cwd='/workspaces/scrapermeleegg'
)
time.sleep(2)
print(f'Server PID: {server.pid}')

r = subprocess.run('curl -sI http://localhost:8080/ | head -10', shell=True, capture_output=True, text=True)
print(r.stdout)
print('Done: http://localhost:8080')

In [ ]:
import subprocess, os
os.chdir('/workspaces/scrapermeleegg')

# Stage all changed files
files = ['generate_site.py', 'server.py', 'launch.py', 'launch.ipynb', 
         'restart.ipynb', 'run_server.ipynb', 'diagnose.py', 'melee_research.ipynb']
for f in files:
    subprocess.run(['git', 'add', f], capture_output=True)

status = subprocess.run(['git', 'status', '--short'], capture_output=True, text=True)
print('Staged files:')
print(status.stdout)

# Commit
commit = subprocess.run(
    ['git', 'commit', '-m', 
     'feat: home panel with tournament cards + API server\n\n'
     '- Replace dropdown selector with full home panel (tournament cards, add box, progress bar)\n'
     '- Add server.py with REST API (add/remove tournaments from browser)\n'
     '- Add active tournament bar for navigation between home and analysis\n'
     '- Add launch.py, restart.ipynb, run_server.ipynb helper scripts\n'
     '- Keyboard shortcuts: 0=home, 1=metagame, 2=matchups, 3=detail'],
    capture_output=True, text=True
)
print('\nCommit:')
print(commit.stdout or commit.stderr)

# Push
push = subprocess.run(['git', 'push', 'origin', 'main'], capture_output=True, text=True, timeout=30)
print('\nPush:')
print(push.stdout or push.stderr)

In [1]:
import subprocess, os
os.chdir('/workspaces/scrapermeleegg')

# Stage files
files = ['generate_site.py', 'server.py', 'launch.py', 'launch.ipynb', 
         'restart.ipynb', 'run_server.ipynb', 'diagnose.py', 'melee_research.ipynb']
for f in files:
    subprocess.run(['git', 'add', f], capture_output=True, text=True)

# Verify staged
status = subprocess.run(['git', 'status', '--short'], capture_output=True, text=True, timeout=10)
print('Staged:')
print(status.stdout)

 M generate_site.py
 M melee_research.ipynb
?? diagnose.py
?? launch.ipynb
?? launch.py
?? restart.ipynb
?? run_server.ipynb
?? server.py


